# Fixed Point Precision Study on MLP-100

This note book is the final stage of the model preparation for benchmarking. The next stage after this notebook is to simply implement the model in C using the knowledge gained from this notebook.
The fixed-point operations defined here tries to simulate the computations performed in SPAR. This might change over-time.

**NOTE:**
The programs/code-snippets in this notebook follows C-like interfaces on purpose.
This is done so that, these code can be easily translated into C for the next stage of study.

**Goals:**
- Load the Model and Dataset from the SQLite3 databases.
- Validate numpy model on the dataset.
- Define Fixed-Point methods.
- Implement model in fixed-point.
- Experiment with Fixed-Point precisions.

# Load Floating-Point Model Parameters and Datset

Here, the model and the dataset exported by the Numpy model extraction notebook is loaded and verified.

In [ ]:
# Delete the cache and import sqlite3 utilities
!rm -rf __pycache__/
from utilsqlite3 import *

## Load the Dataset

In [ ]:
# Load and check the dataset table
Dataset_path = './saved/mnist_test_data-94.00p.s3db'

table_names = getTableNames(Dataset_path)
print('table_names:', table_names)

# Read the header table
header_records =  getRecords(Dataset_path, 'Header')
header_dict = {}
print('')
for r in header_records: 
    print(r[1:3])
    header_dict[r[1]] = r[2]
    
# Get the table names
Data_table = header_dict['dataset.table']
Feature_table = header_dict['features.table']
Label_table = header_dict['labels.table']
print('')
print('Data_table:', Data_table)
print('Feature_table:', Feature_table)
print('Label_table:', Label_table)

In [ ]:
# Read the label_to_index dictionary
labels_records = getRecords(Dataset_path, Label_table)
print('labels_records:', labels_records)

Label_to_index = {label:index for (index, label) in labels_records}
Index_to_label = {index:label for (index, label) in labels_records}
print('Label_to_index:', Label_to_index)
print('Index_to_label:', Index_to_label)

### Build the Dataset Array

In [ ]:
import numpy as np
import dataclasses
from dataclasses import dataclass
from typing import List


# Dataset item class
@dataclass
class DataItem:
    label: str
    label_index: int
    predicted_index: int
    feature_vec: List[np.float32]
        
    def getItemSummary(self):
        return str((self.label, self.label_index, self.predicted_index, self.feature_vec.shape))

In [ ]:
# Read the features and data-item records then merge them
# Make the feature_id:feature_vec map
feat_records = getRecords(Dataset_path, Feature_table)
featid_map = {}
for r in feat_records:
    feat_id = r[0]       # first column is the feature ID
    feat_vec = r[1:]
    featid_map[feat_id]  = feat_vec

    
# Read the data-items and put them in DataItem array
Dataset = []
data_records = getRecords(Dataset_path, Data_table)
data_schema = getColNames(Dataset_path, Data_table)
print('data_schema:', data_schema)

for r in data_records:
    label = r[1]
    label_index = r[2]
    pred_index = r[3]
    feat_id = r[4]
    feat_vec = np.array(featid_map[feat_id], dtype=np.float32)
    item = DataItem(label, label_index, pred_index, feat_vec)
    Dataset.append(item)

item = Dataset[0]
print('Dataset:', len(Dataset))
print('item:', item.getItemSummary())

In [ ]:
# Delete names to avoid confusion later
del table_names, r, labels_records, header_records, header_dict

## Load the Trained Model

In [ ]:
# Load and check the model parameters table
Model_path = './saved/trained-mlp12-94.00p.s3db'

table_names = getTableNames(Model_path)
print('table_names:', table_names)

# Read the header table
header_records =  getRecords(Model_path, 'Header')
Header_dict = {}
print('')
for r in header_records: 
    print(r[1:3])
    Header_dict[r[1]] = r[2]


# Get the table names for later use
Fc1w_table = Header_dict['fc1.weight.table']
Fc1b_table = Header_dict['fc1.bias.table']
Fc2w_table = Header_dict['fc2.weight.table']
Fc2b_table = Header_dict['fc2.bias.table']

In [ ]:
# Returns a table saved using createMatrixTable as a list of tuples
def readMatrixTable(db_path, table_name):
    # read the records
    rec_list = getRecords(db_path, table_name)
    # build the matrix
    rec_list.sort()         # sort by row_no (first column)
    matrix = []
    for rec in rec_list:
        matrix.append(rec[1:])  # stripe off the row_no columns
    return matrix


# test this functions
mat1 = np.array(readMatrixTable(Model_path, Fc1w_table))
mat1.shape

In [ ]:
# Returns the weights and biases as a dictionary
def readModelParams(db_path, table_names):
    model_params = {}
    for name in table_names:
        # read the matrix as a list of tuples
        mat = readMatrixTable(db_path, name)
        # Check if it is a matrix or a vector
        if len(mat)==1: is_vector = True
        else: is_vector = False
        # convert to numpy array
        if is_vector: mat = np.array(mat[0])    # make a 1D array for vectors
        else: mat = np.array(mat)
        # save it for returning
        model_params[name] = mat
    return model_params


# List of table names for running loop
ParamTable_names = [
    Fc1w_table,
    Fc1b_table,
    Fc2w_table,
    Fc2b_table,
]
print('ParamTable_names:', ParamTable_names)

# Read the model parameters as numpy matrix/vectors
model_params = readModelParams(Model_path, ParamTable_names)
for k, v in model_params.items():
    print(f'{k}:', v.shape, v.dtype)

In [ ]:
# Model Parameters class
@dataclass
class mlp12_Params:
    fc1_weight: np.ndarray
    fc2_weight: np.ndarray
    fc1_bias: np.ndarray
    fc2_bias: np.ndarray
        

# Instantiate the model parameter class with float32 datatype
Model_params = mlp12_Params(
    model_params[Fc1w_table].astype(np.float32),
    model_params[Fc2w_table].astype(np.float32),
    model_params[Fc1b_table].astype(np.float32),
    model_params[Fc2b_table].astype(np.float32),
)

# Show the parameter info
for field in dataclasses.fields(Model_params):
    field_value = getattr(Model_params, field.name)
    print(field.name+':', field_value.shape, field_value.dtype)

In [ ]:
del feat_id, feat_records, feat_vec, featid_map, field, field_value
del item, k, label, label_index, mat1, pred_index, r, v, table_names

# Verify Model on the Dataset

In [ ]:
# Relu on numpy array
def npReLU(np_arr):
    return np.maximum(0, np_arr)


# Performs the forward inference on the mlp12 model
# params: model parameters, an instance of mlp12_Params
def mlp12_forward(params, feature_vec):
    x1 = params.fc1_weight @ feature_vec + params.fc1_bias
    fc1_out = npReLU(x1)
    fc2_out = params.fc2_weight @ fc1_out + params.fc2_bias
    return fc2_out


# Uses the forward pass and converts the result into predicted_index
def mlp12_predict(params, feature_vec):
    out_vec = mlp12_forward(params, feature_vec)
    return np.argmax(out_vec)   # return the index of the highest probable class


# Test this model
item = Dataset[0]
pred = mlp12_predict(Model_params, item.feature_vec)
print('pred:', pred)
print(item.getItemSummary())

In [ ]:
from tqdm.auto import tqdm


# Validate the Given model on the whole dataset
# model_params: instance of mlp12_Params
# model_predict: a function that takes model_params and a feacture_vec to compute predicted_index
def validateModel(model_params, model_predict):
    expect_miss = 0      # keeps track of no. of mismatches between prediction in dataset vs model prediction
    total_count = 0
    correct_count = 0
    for item in tqdm(Dataset):
        pred_index = model_predict(model_params, item.feature_vec)
        if pred_index != item.predicted_index: expect_miss += 1    # prediction does not match prediction in dataset
        if pred_index == item.label_index: correct_count += 1   # prediction matched the actual label-index
        total_count += 1
    # Compute and print statistics
    accuracy = (100.0 * correct_count) / total_count
    print(f'Validation accuracy: {accuracy:.2f}%   correct_count: {correct_count}   expected-miss: {expect_miss}   total_count: {total_count}')
    return accuracy, correct_count, expect_miss, total_count


validateModel(Model_params, mlp12_predict)
print(f'Expected:  accuracy: {Header_dict["accuracy"]}%', '  correct_count:', Header_dict['correct_count'])

# Define Fixed Point Methods

## Data Structure

In [ ]:
# Delete the cache before importing
!rm -rf __pycache__/
from AK_FixedPoint import *

# Run Unit tests to make sure everything is okay
!python3 unittest_fxp.py

## Math Operations

In [ ]:
#Run unit tests to make sure everything is okay
!python3 unittest_fxp_math.py

## Matrix Operations

In [ ]:
from math import inf as INF


# Performs matrix-vector multiplication and keeps track of error.
# status_obj: instance of fxp_Status to get status back
# Returns the output vector and a tupel with intermediate results for debugging: resutl, (...)
def fxp_matmul_mv(fxp_mat, fxp_vec, status_obj=None, debug=False):
    # Make sure all assumptions are met
    assert len(fxp_mat._data.shape) == 2, "fxp_mat must be built from a 2D Numpy array"
    assert len(fxp_vec._data.shape) == 1, "fxp_mat must be built from a 1D Numpy array"
    assert fxp_mat._data.shape[1] == fxp_vec._data.shape[0], "Matrix column count not equal vector length"
    
    # Get the data-type parameters
    t_width = fxp_vec._total_width
    f_width = fxp_vec._frac_width
    compute_status = True if status_obj != None else False
    
    # multiply row-wise
    prod_np = (fxp_mat._data * fxp_vec._data)   # multiplying raw values
    # compute error status for multiplying into 2x wider result (less likely to have errors in this step)
    fxp_prod = fxp_makeWider(fxp_mat, 2)  # build 2x wider fxp object
    fxp_prod._data = prod_np              # copy the raw product values
    prod_stat = fxp_fitData(fxp_prod, compute_status)   # now fit within this precision
    if compute_status: 
        if debug: print('prod_stat:', prod_stat)
        fxp_accumulateStatus(status_obj, prod_stat)  # record the multiplication errors
        
    # Now scale down to original precision before accumulation; record error status
    prod_np_down = prod_np >> f_width       # discard lower fraction bits
    fxp_prod_down = fxp_makeSame(fxp_mat)   # fxp object with original precision
    fxp_prod_down._data = prod_np_down
    prod_down_stat = fxp_fitData(fxp_prod_down, compute_status)
    if compute_status: 
        if debug: print('prod_down_stat:', prod_down_stat)
        fxp_accumulateStatus(status_obj, prod_down_stat)  # accumulate the scaling errors
        
    # accumulate along rows; record error status
    accum_np = np.sum(fxp_prod_down._data, axis=1)
    fxp_accum = fxp_makeSame(fxp_vec)
    fxp_accum._data = accum_np
    accum_stat = fxp_fitData(fxp_accum, compute_status)
    if compute_status: 
        if debug: print('accum_stat:', accum_stat)
        fxp_accumulateStatus(status_obj, accum_stat)  # accumulate the scaling errors
    return fxp_accum, (prod_np, fxp_prod, prod_np_down, fxp_prod_down, accum_np, fxp_accum)
    
    
    

In [ ]:
# Test
mat_inp = [
    [1, 2, 3, 4],
    [2, 5, 7, 2],
    [9, 3, 5, 0],
]
vec_inp = [4, 8, 1, 2]
mat_np = np.array(mat_inp)
vec_np = np.array(vec_inp)
res_np = mat_np @ vec_np
print(res_np)

total_width = 10
frac_width = 4
stat = fxp_Status(False, 0, -INF, INF, -INF, INF)
fxp_mat_inp, _ = fxp_ctor(total_width, frac_width, mat_np)
fxp_vec_inp, _ = fxp_ctor(total_width, frac_width, vec_np)
fxp_result, dbg = fxp_matmul_mv(fxp_mat_inp, fxp_vec_inp, stat, debug=True)


print('Overall status:', stat)

print('')
fxp_printInfo(fxp_result)
fxp_printValue(fxp_result)

In [ ]:
# Check the intermediate results
prod_np, fxp_prod, prod_np_down, fxp_prod_down, accum_np, fxp_accum = dbg
print(mat_np * vec_np)
print('')

print(prod_np >> (2*frac_width))

print('')
fxp_printInfo(fxp_prod)
fxp_printValue(fxp_prod)

print('')
fxp_printInfo(fxp_prod_down)
fxp_printValue(fxp_prod_down)


print('')
fxp_printInfo(fxp_accum)
fxp_printValue(fxp_accum)


In [ ]:
del accum_np, data_records, data_schema, frac_width, header_records
del item, mat_inp, mat_np, model_params, pred, prod_np, prod_np_down
del res_np, stat, total_width, vec_inp, vec_np, 

# Implement Model in Fixed-Point

In [ ]:
# Convert Model Parameters to fixed point
Fxp_total_width = 30   # don't use 32
Fxp_frac_width = 15

def convertParamsFxp(params, total_width, frac_width):
    fc1w_fxp, stat = fxp_ctor(total_width, frac_width, params.fc1_weight)
    assert stat.overflow==False, "Overflow"
    fc2w_fxp, stat = fxp_ctor(total_width, frac_width, params.fc2_weight)
    assert stat.overflow==False, "Overflow"

    fc1b_fxp, stat = fxp_ctor(total_width, frac_width, params.fc1_bias)
    assert stat.overflow==False, "Overflow"
    fc2b_fxp, stat = fxp_ctor(total_width, frac_width, params.fc2_bias)
    assert stat.overflow==False, "Overflow"
    
    fxp_params = mlp12_Params(fc1w_fxp, fc2w_fxp,  fc1b_fxp, fc2b_fxp)
    return fxp_params


Fxp_model_param = convertParamsFxp(Model_params, Fxp_total_width, Fxp_frac_width)
fxp_printInfo(Fxp_model_param.fc1_weight)
Fxp_model_param.fc1_weight._data

In [ ]:
# Relu on fxp
def fxpReLU(fxp_num):
    relu_out = np.maximum(0, fxp_num._data)
    fxp_ret = fxp_makeSame(fxp_num)
    fxp_ret._data = relu_out
    stat = fxp_fitData(fxp_ret, True)   # is not necessary for Relu
    assert stat.overflow==False, "Unexpected overflow in ReLU"
    return fxp_ret


# Test fxpReLU()
inp_vec = [0, -1, 2, 4, -2]
out_vec = [0,  0, 2, 4,  0]
fxp_test, _ = fxp_ctor(Fxp_total_width, Fxp_frac_width, inp_vec)
relu_out = fxpReLU(fxp_test)
fxp_printValue(relu_out)
result = fxp_getAsFloat(relu_out) == out_vec
assert result.all(), "Problem with fxpReLU()"

In [ ]:
# Performs the forward inference on the mlp12 model
# params: model parameters, an instance of mlp12_Params with fixed-point numbers
def mlp12_forward_fxp(params, feature_vec, debug=False):
    # Make status objects
    stat = fxp_Status(False, 0, -INF, INF, -INF, INF)
    temp_stat = fxp_Status(0,0,0,0,0,0)
    # Layer-1
    #fxp_printInfo(params.fc1_weight)
    #fxp_printInfo(feature_vec)
    x1, _ = fxp_matmul_mv(params.fc1_weight, feature_vec, stat)
    if debug: print(stat)
    x1 = fxp_add(x1, params.fc1_bias, temp_stat)
    fxp_accumulateStatus(stat, temp_stat)
    if debug: print(stat)
    fc1_out = fxpReLU(x1)
    
    # output layer
    x2, _ = fxp_matmul_mv(params.fc2_weight, fc1_out, stat)
    if debug: print(stat)
    fc2_out = fxp_add(x2, params.fc2_bias, temp_stat)
    fxp_accumulateStatus(stat, temp_stat)
    if debug: print(stat)
    return fc2_out, stat


# Uses the forward pass and converts the result into predicted_index
def mlp12_predict_fxp(params, feature_vec, debug=False):
    out_vec, stat = mlp12_forward_fxp(params, feature_vec, debug=debug)
    return np.argmax(out_vec._data), stat   # return the index of the highest probable class and error status


# Test this model
item = Dataset[0]
feat_vec, stat = fxp_ctor(Fxp_total_width, Fxp_frac_width, item.feature_vec)
pred, status = mlp12_predict_fxp(Fxp_model_param, feat_vec , debug=True)
print('pred:', pred)
print(item.getItemSummary())

In [ ]:
# Convert dataset into fixed-point
def convertDatasetFxp(dataset, total_width, frac_width):
    fxp_dataset = []
    for item in dataset:
        fxp_feat_vec, stat = fxp_ctor(total_width, frac_width, item.feature_vec)
        fxp_item = DataItem(item.label, item.label_index, item.predicted_index, fxp_feat_vec)
        fxp_dataset.append(fxp_item)
    return fxp_dataset


# Test
Fxp_Dataset = convertDatasetFxp(Dataset, Fxp_total_width, Fxp_frac_width)
print('Fxp_Dataset:', len(Fxp_Dataset))
item = Fxp_Dataset[0]
pred, status = mlp12_predict_fxp(Fxp_model_param, item.feature_vec , debug=False)
print('pred:', pred, '   Expected:', item.predicted_index)

In [ ]:
# Validate the Fixed-point model on the whole dataset
# model_params: instance of mlp12_Params
# model_predict: a function that takes model_params and a feacture_vec to compute predicted_index
def validateModelFxp(fxp_dataset, model_params, model_predict):
    expect_miss = 0      # keeps track of no. of mismatche between prediction in dataset vs model prediction
    total_count = 0
    correct_count = 0
    stat_list = []
    for item in tqdm(fxp_dataset):
        pred_index, stat = model_predict(model_params, item.feature_vec, debug=False)
        if stat.overflow: stat_list.append(stat)
        if pred_index != item.predicted_index: expect_miss += 1    # prediction does not match prediction in dataset
        if pred_index == item.label_index: correct_count += 1   # prediction matched the actual label-index
        total_count += 1
    # Compute and print statistics
    accuracy = (100.0 * correct_count) / total_count
    print(f'Validation accuracy: {accuracy:.2f}%   correct_count: {correct_count}   expected-miss: {expect_miss}   total_count: {total_count}')
    print(f'Item overflow count: {len(stat_list)}')
    return accuracy, correct_count, expect_miss, total_count, stat_list


validateModelFxp(Fxp_Dataset, Fxp_model_param, mlp12_predict_fxp)
print(f'Expected:  accuracy: {Header_dict["accuracy"]}%', '  correct_count:', Header_dict['correct_count'])

In [ ]:
del status, stat, result, relu_out, pred, Fxp_Dataset
del out_vec, item, inp_vec, fxp_vec_inp, dbg

# Experiment with Fixed-Point Precisions

In [ ]:
# Set up precision to convert model and dataset
trial_int_width = 15
trial_frac_width = 15
trial_total_width = trial_frac_width + trial_int_width

fxp_model_param = convertParamsFxp(Model_params, trial_total_width, trial_frac_width)
fxp_printInfo(fxp_model_param.fc1_weight)

print('')
fxp_dataset = convertDatasetFxp(Dataset, trial_total_width, trial_frac_width)
print('fxp_Dataset:', len(fxp_dataset))
item = fxp_dataset[0]
fxp_printInfo(item.feature_vec)

In [ ]:
# Run validation, collect result, then tabulate
validation_result = validateModelFxp(fxp_dataset, fxp_model_param, mlp12_predict_fxp)
print(f'Expected:  accuracy: {Header_dict["accuracy"]}%', '  correct_count:', Header_dict['correct_count'])

In [ ]:
# Print results for Tabulation
accuracy, correct_count, expect_miss, total_count, stat_list = validation_result
overflow_item_count = len(stat_list)
overflow_count = 0
for stat in stat_list: overflow_count += stat.overflow_count

#row_header = "total_width,  frac_width,  int_width,  accuracy%,  total-items,  correct_count,  prediction-mismatch,  overflow-item-count,  overflow-count"
row_template = "{total_width}\t{int_width}\t{frac_width}\t{accuracy}\t{total_items}\t{correct_count}\t{prediction_mismatch}\t{overflow_item_count}\t{overflow_count}"

print(row_template.replace('\t', ' '))
print(row_template.format(
    total_width=trial_total_width, frac_width=trial_frac_width, int_width=trial_int_width,
    accuracy=accuracy, 
    total_items=len(fxp_dataset), correct_count=correct_count, prediction_mismatch=expect_miss, 
    overflow_item_count=overflow_item_count, overflow_count=overflow_count
))

In [ ]:
# Check the individual item overflow status if needed
max_overflow_count = -1
for stat in stat_list: 
    print(stat)
    max_overflow_count = max(max_overflow_count, stat.overflow_count)

avg_overflow_count = overflow_count / len(stat_list)
print('')
print('max:', max_overflow_count,  '  avg_overflow_count:', avg_overflow_count)